# Initial AI Agent PPO w CNN (working) MS HS Disc

In [21]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [22]:
%cd "J:\git\TensorFlowPSX\Py"

from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 20000
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'img_hist_len': 3,
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 3, # 1 raw image, 2 resized image, 3 greyscale
  'agent' : 'PPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 1 is High Speed Ring x MR2, 2 is 0-400m x MR2, 3 is 0-400 Supra disc, 4 is 0-400 Supra cont, 
}

j:\git\TensorFlowPSX\Py


In [23]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [24]:
import ray
ray.shutdown()
ray.init()

2023-10-05 22:15:00,933	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [26]:
from ray.rllib.algorithms.ppo import PPOConfig

algo = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=128,
        )
    .model(
        post_fcnet_hiddens = [24,24],
        fcnet_activation = "relu",
        )
    .build()
)

#PPOConfig.framework("")

2023-10-05 22:15:35,817	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.


TypeError: 'dict' object is not callable

In [ ]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=16036) reset triggered
(RolloutWorker pid=16036) reload save for track : 2
(RolloutWorker pid=16036) [0. 0.]


(RolloutWorker pid=16036) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\gymnasium\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
(RolloutWorker pid=16036)   if not isinstance(terminated, (bool, np.bool8)):


(RolloutWorker pid=16036) 18 9.0 False
(RolloutWorker pid=16036) [ 0.19110146 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.         0.35475707]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.         0.17816925]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.         0.92991841]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.12847781 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.0976893  0.49379253]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.02924454 0.27680767]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0. 1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0. 1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.        -0.2084105]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.17736638 -0.20160764]
(RolloutWorker pid=16036) 18 0.0 False
(Rollout

(RolloutWorker pid=16036) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.45411590000594515
(RolloutWorker pid=16036)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.3159284 1.       ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.19126752 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.63256848 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.28898811 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.24661463 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.92492002 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.09752855 0.04862738]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.26107281 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0. 1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.15326363 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.5738762 1.       ]
(RolloutWorker pid=16036) 18 0.0 Fals

...

(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.77050006 0.79594445]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.73018992 -0.85146189]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.40761036 -0.22215366]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.00127205 0.84861636]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.01546693 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.60820824 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.41776258 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.48948237 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0. 1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0. -1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.06725025 -0.94992852]
(RolloutWorker pid=16036) 18 0.0 False


(RolloutWorker pid=16036) c:\Users\nadir\anaconda3\envs\GTRay2.6.3_torch\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.41366900000593887
(RolloutWorker pid=16036)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.94119334 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.19907317 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.27985141 -0.45424002]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.31685379 0.98464596]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.21049345 -0.01571119]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.82655978 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0. 1.]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.53300929 -0.68813664]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.42568856 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.         -0.83848369]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.3750352  -0.37838167]
(RolloutWorker pid=1603

KeyboardInterrupt: 

(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.3972722  -0.44753516]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.35585117 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.40261602 0.4808507 ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.48336977 0.26282632]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.70032239 -1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.16446155 1.        ]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.55661809 -0.11052263]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [0.         0.09828305]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.17989409 -0.63331997]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.3321473 -0.0381074]
(RolloutWorker pid=16036) 18 0.0 False
(RolloutWorker pid=16036) [ 0.         -0.17926192]
(RolloutWor